In [1]:
from tomok.ifc import IFCReader

In [2]:
IFCReader(ifc_filepath="/Users/jaewooklee/Documents/Github/tomok/tomok/ED_extension_usingToolkit_v2.ifc").get_products()

{'entity_instances': [[95/IfcBuilding/] [pset: ] [mat: ] [prof: ] [rule:None]], 'IfcBuilding': {'entity_instances': [[4382780/IfcRelAggregates/] [pset: None] [mat: None] [prof: None] [rule:None], [4382792/IfcRelDefinesByProperties/] [pset: Pset_BuildingSystemCommon] [mat: None] [prof: None] [rule:None]], 'pset': {'Pset_BuildingSystemCommon': #4382787=IfcPropertySet('00Np4gWQGC_NmpLzsxSTna',#18,'Pset_BuildingSystemCommon',$,(#125))}, 'material': {}, 'profile': {}, 'id2rel_class': {}, 'rule_decision': [], 'rule_indirect': []}, 'pset': {}, 'material': {}, 'profile': {}, 'id2rel_class': {}, 'rule_decision': [], 'rule_indirect': []}
currently IfcCountMeasure (5, <class 'int'>) type is not handled in IFCReader
[(#95=IfcBuilding('0UiHtyh21CPeXMlHYS8V49',#18,'',$,$,#93,$,'',.ELEMENT.,$,$,#94), []), (#4382780=IfcRelAggregates('0$GOtzgqo4MWyllY8ehasb',#18,$,$,#124,(#95)), [#95=IfcBuilding('0UiHtyh21CPeXMlHYS8V49',#18,'',$,$,#93,$,'',.ELEMENT.,$,$,#94)]), (#4382792=IfcRelDefinesByProperties('2yXC

Exception: 